In [1]:
import sys
sys.path.append('../input/pytorch-images-seresnet')

import os
import math
import time
import random
import shutil
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict, Counter

import scipy as sp
import numpy as np
import pandas as pd

from tqdm.auto import tqdm
from functools import partial

import cv2
from PIL import Image

from matplotlib import pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
import albumentations
from albumentations import *
from albumentations.pytorch import ToTensorV2


import timm

from torch.cuda.amp import autocast, GradScaler

import warnings 
warnings.filterwarnings('ignore')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
IMAGE_SIZE = 640
BATCH_SIZE = 128

TEST_PATH = '../input/ranzcr-clip-catheter-line-classification/test'
RES_MODEL_PATH = '../input/resnet200d-public/resnet200d_320_CV9632.pth'


test = pd.read_csv('../input/ranzcr-clip-catheter-line-classification/sample_submission.csv')

In [3]:
class TestDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['StudyInstanceUID'].values
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        file_path = f'{TEST_PATH}/{file_name}.jpg'
        image = cv2.imread(file_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        return image

In [4]:
def get_transforms():
        return Compose([
            Resize(IMAGE_SIZE, IMAGE_SIZE),
            Normalize(
            ),
            ToTensorV2(),
        ])

In [5]:
class ResNet200D(nn.Module):
    def __init__(self, model_name='resnet200d_320'):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=False)
        n_features = self.model.fc.in_features
        self.model.global_pool = nn.Identity()
        self.model.fc = nn.Identity()
        self.pooling = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(n_features, 11)

    def forward(self, x):
        bs = x.size(0)
        features = self.model(x)
        pooled_features = self.pooling(features).view(bs, -1)
        output = self.fc(pooled_features)
        return output

In [6]:
res_model = ResNet200D()
res_model.load_state_dict(torch.load(RES_MODEL_PATH)['model'])

<All keys matched successfully>

In [7]:
models = [res_model.to(device)]

In [9]:
res_model.eval()

ResNet200D(
  (model): ResNet(
    (conv1): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
      (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    )
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act1): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=T

In [10]:
def inference(models, test_loader, device):
    tk0 = tqdm(enumerate(test_loader), total=len(test_loader))
    probs = []
    for i, (images) in tk0:
        images = images.to(device)
        avg_preds = []
        for model in models:
            with torch.no_grad():
                y_preds1 = model(images)
                y_preds2 = model(images.flip(-1))
            y_preds = (y_preds1.sigmoid().to('cpu').numpy() + y_preds2.sigmoid().to('cpu').numpy()) / 2
            avg_preds.append(y_preds)
        avg_preds = np.mean(avg_preds, axis=0)
        probs.append(avg_preds)
    probs = np.concatenate(probs)
    return probs

In [11]:
test_dataset = TestDataset(test, transform=get_transforms())
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, 
                         num_workers=4 , pin_memory=True)
predictions = inference(models, test_loader, device)

  0%|          | 0/28 [00:00<?, ?it/s]

In [12]:
target_cols = test.iloc[:, 1:12].columns.tolist()
test[target_cols] = predictions

test.head()

StudyInstanceUID  ETT - Abnormal  \
0  1.2.826.0.1.3680043.8.498.46923145579096002617...        0.042606   
1  1.2.826.0.1.3680043.8.498.84006870182611080091...        0.000010   
2  1.2.826.0.1.3680043.8.498.12219033294413119947...        0.000016   
3  1.2.826.0.1.3680043.8.498.84994474380235968109...        0.001823   
4  1.2.826.0.1.3680043.8.498.35798987793805669662...        0.000026   

   ETT - Borderline  ETT - Normal  NGT - Abnormal  NGT - Borderline  \
0          0.621123      0.217875        0.000425          0.003781   
1          0.000032      0.000242        0.000163          0.000110   
2          0.000017      0.000109        0.000139          0.000109   
3          0.020463      0.044035        0.022296          0.011816   
4          0.000189      0.001200        0.001190          0.000670   

   NGT - Incompletely Imaged  NGT - Normal  CVC - Abnormal  CVC - Borderline  \
0                   0.011037      0.980665        0.037586          0.140550   
1                   0.000041      0.000067        0.004692          0.004040   
2                   0.000012      0.000033        0.005357          0.241475   
3                   0.919738      0.031636        0.049662          0.051236   
4                   0.000095      0.002048        0.003164          0.006033   

   CVC - Normal  Swan Ganz Catheter Present  
0      0.889836                    0.999464  
1      0.998147                    0.000002  
2      0.770350                    0.000017  
3      0.913974                    0.000491  
4      0.991957                    0.000002